Se quiere saber qué naciones tienen mayor indice de conversión, utilizando duración de sesión, número de páginas que ha visto el usuario, y el dispositivo que estaban utilizando

In [1]:
#Se importan las librerias
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [2]:
#Se inicia sesion de spark
spark = SparkSession.builder.appName('UsuarioWeb').getOrCreate()

In [3]:
#Se crea el df
df = spark.read.json('hdfs://namenode:8020/user/hadoop/usuarios_web.json', multiLine=True)

In [5]:
df.printSchema()

root
 |-- conversion: boolean (nullable = true)
 |-- dispositivo: string (nullable = true)
 |-- duracion_sesion: double (nullable = true)
 |-- paginas_vistas: long (nullable = true)
 |-- pais: string (nullable = true)
 |-- referencia: string (nullable = true)
 |-- usuario_id: string (nullable = true)



In [7]:
#Se calcula el promedio de duracion de sesion por pais
df.groupBy('pais').agg(avg('duracion_sesion').alias('promedio duracion')).orderBy('promedio duracion', ascending=False).show()

+--------------+------------------+
|          pais| promedio duracion|
+--------------+------------------+
|Estados Unidos|33.400689655172414|
|        España| 31.55228571428572|
|        Canadá|31.235945945945947|
|      Colombia|30.105937500000007|
|     Argentina| 29.90222222222223|
|        México|24.219250000000006|
+--------------+------------------+



In [8]:
#Se calcula el promedio de duracion de sesion por dispositivo
df.groupBy('dispositivo').agg(avg('duracion_sesion').alias('promedio duracion')).orderBy('promedio duracion', ascending=False).show()

+-----------+------------------+
|dispositivo| promedio duracion|
+-----------+------------------+
|      Móvil|34.576451612903234|
|Computadora|28.008695652173913|
|     Tablet|27.418260869565213|
+-----------+------------------+



In [9]:
#Se calcula el promedio de paginas vistas por dispositivo
df.groupBy('dispositivo').agg(avg('paginas_vistas').alias('promedio paginas vistas')).orderBy('promedio paginas vistas', ascending=False).show()

+-----------+-----------------------+
|dispositivo|promedio paginas vistas|
+-----------+-----------------------+
|      Móvil|      8.790322580645162|
|Computadora|       8.36231884057971|
|     Tablet|      7.521739130434782|
+-----------+-----------------------+



In [10]:
#Se crea un df solo con los usuarios convertidos y se calcula el promedio de duracion de sesion y paginas vistas
conv = df.filter(col('conversion') == True)
conv.select(avg('duracion_sesion').alias('promedio duracion sesion'), avg('paginas_vistas').alias('promedio paginas vistas')).show()

+------------------------+-----------------------+
|promedio duracion sesion|promedio paginas vistas|
+------------------------+-----------------------+
|      31.155980392156863|      8.470588235294118|
+------------------------+-----------------------+



In [11]:
#De los usuarios convertidos se agrupan por pais
conv.groupBy("pais").count().orderBy("count", ascending=False).show()

+--------------+-----+
|          pais|count|
+--------------+-----+
|        Canadá|   20|
|        México|   20|
|Estados Unidos|   17|
|      Colombia|   17|
|        España|   15|
|     Argentina|   13|
+--------------+-----+



In [12]:
#Se agrupa por referencia los usuarios convertidos
conv.groupBy("referencia").count().orderBy("count", ascending=False).show()

+--------------+-----+
|    referencia|count|
+--------------+-----+
|       Anuncio|   27|
|      Orgánico|   26|
|       Directo|   25|
|Redes Sociales|   24|
+--------------+-----+



La tasa de fuente de tráfico que genera más conversiones son los Anuncios y Orgánico.
Aunque México cuenta con el menor tiempo promedio de duración de sesión, empata con Canadá por el país con más conversiones. Esto indica que la población de México es más susceptible a realizar alguna acción dentro de la página web.
Con esta información se podría tomar el territorio mexicano como un buen lugar para probar campañas publicitarias antes de sacarles globalmente.